In [1]:
import pandas as pd
import numpy as np
from nltk import *
import pickle
import re
from nltk.corpus import wordnet

In [2]:
# Load our data and try
with open('modified_data/just_tags.txt', 'rb') as f:
    just_tags = pickle.load(f)
    
with open('modified_data/just_words.txt', 'rb') as f:
    just_words = pickle.load(f)

In [3]:
# Make hindi dictionary from training data becuase theres no phonetically typed hindi dictionary anywhere
hin_dict = [w for i, w in enumerate(just_words) if just_tags[i] == 'HI']

In [4]:
# How wordnets dictionary works:
# Case insensitive so its good for our data
if wordnet.synsets('lIbRary'):
    print('ENGLISH')
else:
    print('NOT ENGLISH')

ENGLISH


In [5]:
# Just check if emoticons with alphabets are present in the training set
':P' in just_words

False

In [6]:
# How do emoticons look?
[w for i, w in enumerate(just_words) if just_tags[i] == 'EMT']

['😡',
 ':)',
 ';)',
 ':)',
 ';)',
 ':)',
 ':)',
 ':)',
 ':)',
 ':(',
 ';)',
 ':)',
 ':)',
 ';)',
 ';)',
 ':)',
 ':)',
 ':)',
 ':)',
 ':-)',
 '😜',
 ':(',
 ':)',
 ':)',
 ':(',
 ':)',
 ':-)',
 ':)',
 ':)',
 ':)',
 ':/',
 ':(',
 ';)',
 ':)',
 ':-(',
 ':/',
 ':@',
 ':@',
 ':(',
 ':-)',
 ';)',
 ':/',
 ':(',
 ':(',
 ';)',
 ':3',
 ':)',
 ':-)',
 ':(',
 ':)',
 ':)',
 ':)',
 ':)',
 ':)',
 ';)',
 ':)',
 ':-(',
 ':-)',
 ':)',
 ';)',
 ':-(',
 ':)',
 ':)',
 ':(',
 ':(',
 ':(',
 ':(',
 ';)',
 ':)',
 ':)',
 ':(',
 ':(',
 ':-(',
 ';)',
 ';)',
 ':)',
 ':)',
 ':(',
 ':)',
 ';)',
 ':)',
 ';)',
 ':)',
 ':)',
 '😡',
 ':)',
 ':)',
 ':)',
 ':)',
 ';)',
 ':(',
 ';)',
 ':-/',
 ':)',
 ':)',
 ':)',
 ':)',
 ':)',
 ':(',
 ':(',
 ':(',
 ':)',
 '😘',
 ';)',
 ':)',
 ':)',
 ':-)',
 ';)',
 ':)',
 ':)',
 ':)',
 ';)',
 ':)',
 ':)',
 ':)',
 ':/',
 ':-)',
 ':)',
 ';)',
 ':(',
 ':)',
 ':)',
 ':)',
 ':@',
 ':)',
 ';)',
 ':)',
 ':)',
 ':)',
 ':(',
 ':/',
 ':-(',
 ':)',
 ':-)',
 ':)',
 ':)',
 ':(',
 ':)',
 ':)',
 ';)',
 ';)',
 ':

In [7]:
# The 4 possible tags we need to classifiy for
np.unique(just_tags)

array(['EMT', 'EN', 'HI', 'UN'], dtype='<U3')

In [8]:
# Classifier that uses wordnet's dictionary
def classify_dictionary(words):
    preds = []
    for w in words:
        if wordnet.synsets(w):
            preds.append('EN')
            
        elif re.search('[a-zA-Z]', w) == None:
            preds.append('EMT')
            
        elif w in hin_dict:
            preds.append('HI')
        
        else:
            preds.append('UN')
            
    return preds

In [9]:
preds_dict = classify_dictionary(just_words)

In [10]:
def classification_acc(true_y, pred_y):
    classified = [True for i in range(len(true_y)) if true_y[i] == pred_y[i]]
    return (np.sum(classified)/len(true_y))*100

In [11]:
print ('Classification accuracy using just dictionaries is:', classification_acc(just_tags, preds_dict), '%')

Classification accuracy using just dictionaries is: 55.988719639346215 %
